# Adversarial Training V1

TJ Kim

11.22.21

### Summary:
- Take the FedEM and Locally trained weights from prior and run "X" epochs of localized adversarial training on top of the weights (just for the models used in question)
- Measure Transferability and weights similar to prior iterations

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Libaries

In [4]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.Boundary_Transferer import *

### FedEM Analysis
- Load aggregator, clients
- Perform attack on all datapoints in training set of each local model 
- Perform training on top of the models
- Pass these models through the transferer/Decision boundary classes

Load and generate models

In [5]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_09_28_first_transfers/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]
    
weights = np.load("weights/cifar/21_09_28_first_transfers/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 164.58it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:34<00:00,  2.33it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [17]:
clients[0].train_iterator.dataset.targets

tensor([5, 2, 6, 0, 1, 4, 6, 1, 2, 9, 6, 1, 0, 1, 1, 6, 6, 5, 5, 5, 1, 5, 0, 4,
        6, 9, 9, 2, 9, 5, 5, 1, 2, 5, 4, 2, 3, 6, 9, 6, 9, 6, 9, 6, 9, 1, 6, 2,
        9, 9, 1, 1, 2, 1, 1, 4, 9, 5, 9, 1, 5, 1, 6, 2, 9, 1, 5, 5, 9, 2, 6, 1,
        0, 0, 3, 4, 9, 1, 2, 4, 4, 4, 7, 6, 1, 1, 5, 4, 1, 2, 4, 4, 1, 6, 5, 4,
        0, 2, 0, 0, 4, 2, 6, 6, 9, 9, 6, 5, 0, 4, 1, 4, 5, 2, 6, 4, 0, 9, 1, 5,
        6, 0, 1, 6, 7, 8, 1, 4, 4, 4, 4, 6, 3, 6, 2, 9, 6, 0, 0, 5, 1, 0, 1, 9,
        9, 2, 2, 4, 5, 2, 2, 9, 1, 5, 0, 1, 9, 2, 1, 2, 6, 1, 3, 2, 4, 5, 9, 6,
        2, 0, 5, 4, 1, 2, 9, 0, 9, 0, 2, 5, 2, 9, 1, 4, 5, 0, 9, 9, 5, 0, 1, 5,
        0, 9, 5, 4, 2, 1, 6, 2, 4, 9, 6, 1, 1, 9, 1, 9, 5, 2, 6])